In [1]:
import os

import numpy as np
from statistics import mean
from decimal import Decimal
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import matplotlib.patches as patches
import bamt.networks as Nets
import bamt.preprocessors as pp
import bamt
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path


from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from bamt.builders import StructureBuilder
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import r2_score
import math
import random
from scipy.spatial.distance import cdist
from numpy import array
from pgmpy.utils import get_example_model
from pgmpy.estimators import PC
from pgmpy.estimators import K2Score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from joblib import Parallel, delayed
from pyvis.network import Network
from pyitlib import discrete_random_variable as drv
import bamt.builders as builders
import itertools 
from itertools import islice
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import numpy as np
from bamt.networks.continuous_bn import ContinuousBN
from bamt.networks.hybrid_bn import HybridBN


### Загрузка ранее сформированных датасетов соответсвующих определенному сезону и сектору моря

In [2]:
zima= pd.read_csv('Зима2019.npy_Laptevyh_const100200.csv')
zima =zima.drop('Unnamed: 0', axis=1)

vesna= pd.read_csv('Весна2019.npy_Laptevyh_const100200.csv')
vesna =vesna.drop('Unnamed: 0', axis=1)

leto= pd.read_csv('Лето2019.npy_Laptevyh_const100200.csv')
leto =leto.drop('Unnamed: 0', axis=1)

osen= pd.read_csv('Осень2019.npy_Laptevyh_const100200.csv')
osen =osen.drop('Unnamed: 0', axis=1)

In [3]:
df = pd.concat([zima, vesna, leto, osen], ignore_index= True)

In [4]:
df=df.astype({'NORM': 'float64'})
data_train, data_test = train_test_split(df, test_size=0.3, random_state=42)
target = data_test['NORM'].values
data_test = data_test.drop(columns=['NORM'])

### Применение функции для обучения Байесовской сети на ранее размеченных данных с искусственно сгенерированными аномалиями

In [5]:
from anomaly_search_functions import bn_learning

In [6]:
bn1=bn_learning(data_train)

#### Сохранение БС для дальнейшего использования

In [7]:
bn1.save("Laptevyh_sektor_bn.json")

True

### Демонстрация поиска аномалий с применением ранее обученной Байесовской сети на тестовых данных с искусственно сгенерированными аномалиями

In [5]:
bn2 = HybridBN(use_mixture=True, has_logit=True)

bn2.load("Laptevyh_sektor_bn.json")

In [6]:
from anomaly_search_functions import anom_search_func

In [7]:
anomalii = anom_search_func(data_test,bn2,1)

100%|█████████████████████████████████████████████████████████████████████████████| 5460/5460 [00:14<00:00, 378.02it/s]


In [9]:
f1_score(target,anomalii, average='macro') 

0.8143204711646759